<a href="https://colab.research.google.com/github/arquansa/PSTB-exercises/blob/main/W7D4DC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pinecone Serverless Reranking Last Updated: April 27th, 2025

#Daily Challenge: Pinecone Serverless Reranking in Action


Why are we doing this?
Reranking models boost search relevance by assigning similarity scores between a query and documents, then reordering results so the most pertinent information appears first. In contexts like healthcare, this helps clinicians quickly access the most critical clinical notes.



Task Overview & Detailed Explanations
Below is a skeleton pipeline. Each numbered item is an action you must complete. After every instruction, you’ll find a clear explanation of what to do and why it’s important. Whenever you see ..., replace it with the appropriate code or value, using the hint for guidance.



Part 1: Load Documents & Execute Reranking Model


1. Install Pinecone libraries



pip install pinecone==6.0.1 pinecone-notebooks


What to do: Run this command in your terminal or notebook to install the Pinecone client library and the notebook helper package.
Why: You’ll need the client package to interact with Pinecone’s API and the notebook helper to simplify authentication in environments like Colab.


2. Authenticate with Pinecone



import os
if not os.environ.get("PINECONE_API_KEY"):
   from pinecone_notebooks.colab import Authenticate
   Authenticate()


What to do: Check if your environment has the PINECONE_API_KEY. If not, call Authenticate() to prompt for it.
Why: Securely providing your API key lets the client connect to your Pinecone project without hard-coding secrets in your script.


3. Instantiate the Pinecone client



from pinecone import Pinecone
api_key = os.environ["PINECONE_API_KEY"]
environment = "..."  # e.g., "us-west1-gcp"
pc = Pinecone(api_key=api_key, environment=environment)


What to do: Fill in your Pinecone project’s environment string (found in your Pinecone dashboard) in place of .... Then create a Pinecone client instance.
Why: The client (pc) is your entry point for all Pinecone operations—creating indexes, querying, and reranking.


4. Define your query & documents



query = "Tell me about Apple's products"
documents = [
   ...  # Provide five text strings: some about the fruit, some about the company
]


What to do: Replace ... with a list of five example sentences that include both references to the fruit “apple” and the company “Apple Inc.”.
Why: You need a small set of documents to test the reranker’s ability to distinguish between different contexts of the same word.


5. Call the reranker



from pinecone import RerankModel
reranked = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=query,
   documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
   top_n=...  # e.g., 3
)


What to do: Fill in top_n with how many top results you want returned (e.g., 3).
Why: top_n limits the number of reranked results, so you only retrieve the most relevant documents.


6. Inspect reranked results



def show_reranked(query, matches):
   print(f"Query: {query}")
   for i, m in enumerate(matches):
       ...  # Print the position (i+1), m.score, and m.document.text
show_reranked(query, reranked.matches)


What to do: Replace ... with code that prints out the rank (i+1), the similarity score m.score, and the document text m.document.text.
Why: Seeing these values demonstrates how the reranker orders documents and what scores it assigns.


Part 2: Setup a Serverless Index for Medical Notes
1. Install data & model libraries



pip install pandas torch transformers


What to do: Install pandas for data manipulation, torch for model inference, and transformers for loading embedding models.
Why: You’ll use these libraries to load, embed, and manipulate medical note data.


2. Import modules & define environment settings



import os, time, pandas as pd, torch
from pinecone import Pinecone, ServerlessSpec

cloud = "..."        # e.g., "aws"
region = "..."       # e.g., "us-east-1"
spec = ServerlessSpec(cpu=..., memory_gb=...)
index_name = "pinecone-reranker"

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], environment=f"{cloud}-{region}")


What to do: Fill in cloud and region with your Pinecone project’s deployment environment. Choose CPU and memory values in ServerlessSpec.
Why: You’re configuring a serverless index tailored to your resource requirements and connecting the client in the proper cloud region.


3. Create or recreate the index



if pc.has_index(index_name):
   pc.delete_index(index_name)
pc.create_index(
   name=index_name,
   dimension=...,           # must match embedding vector size
   serverless_config=spec
)


What to do: Set dimension equal to your embedding model’s output size (e.g., 384).
Why: The index’s dimension must match the embedding vectors you’ll insert, otherwise upserts will fail.


Part 3: Load the Sample Data
1. Download & read JSONL



import requests, tempfile

with tempfile.TemporaryDirectory() as tmpdir:
   file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")
   url = "..."  # raw GitHub URL to JSONL file
   resp = requests.get(url)
   resp.raise_for_status()
   open(file_path, "wb").write(resp.content)
   df = pd.read_json(file_path, orient='records', lines=True)


What to do: Insert the raw URL in place of ... to download the sample medical notes.
Why: You need a DataFrame of medical notes (with embeddings already available) to index and test queries.


2. Preview the DataFrame



print(df.head())


What to do: Run this to view the first few rows of the DataFrame.
Why: Ensures you have the right columns (e.g., id, embedding, metadata) before upserting.


Part 4: Upsert Data into the Index
1. Instantiate index client & upsert



index = pc.Index(index_name)
index.upsert_from_dataframe(df)


What to do: Create an Index object and call upsert_from_dataframe.
Why: This pushes all your note embeddings and metadata into Pinecone for later queries.


2. Wait for availability



def is_ready(idx):
   stats = idx.describe_index_stats()
   return stats.total_vector_count > 0

while not is_ready(index):
   time.sleep(5)
print(index.describe_index_stats())


What to do: Poll until total_vector_count is greater than zero.
Why: Ensures that upserted vectors are fully indexed before you attempt to query.


Part 5: Query & Embedding Function
1. Define your embedding function



from sentence_transformers import SentenceTransformer

def get_embedding(text):
   model = SentenceTransformer("...")  # e.g., "all-MiniLM-L6-v2"
   return model.encode(text)


What to do: Provide the name of the sentence-transformer model you plan to use in place of ....
Why: Converts incoming queries into the same vector space as your indexed notes.


2. Run a semantic search query



question = "..."  # e.g., "what if my patient has leg pain"
emb = get_embedding(question)
results = index.query(vector=emb, top_k=..., include_metadata=True)
matches = sorted(results.matches, key=lambda m: m.score, reverse=True)


What to do: Replace question, set top_k for number of results (e.g., 5).
Why: Retrieves the most semantically similar notes from the index based on your clinical query.


Part 6: Display & Rerank Clinical Notes
1. Display initial search results



def show_results(q, matches):
   print(f"Question: {q}")
   for i, m in enumerate(matches):
       ...  # print i+1, m.id, m.score, m.metadata
show_results(question, matches)


What to do: Fill in the print statement to show rank, vector ID, similarity score, and metadata.
Why: Helps you see which notes were initially considered most relevant.


2. Prepare documents for reranking



rerank_docs = [
   {"id": m.id, "reranking_field": "; ".join([f"{k}: {v}" for k, v in m.metadata.items()])}
   for m in matches
]
rerank_query = "..."  # e.g., a more specific clinical question


What to do: Set rerank_query to a refined question that tests finer distinctions (e.g., focusing on a procedure or symptom).
Why: Constructs a field summarizing each note’s metadata for the reranker to use when rescoring.


3. Execute serverless reranking



reranked = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=rerank_query,
   documents=rerank_docs,
   rank_fields=["reranking_field"],
   top_n=...  # number of top reranked notes to view
)


What to do: Choose top_n to specify how many reranked results you need.
Why: Reranking uses the refined query and metadata field to reorder notes by their new relevance scores.


4. Show reranked results



def show_reranked(q, matches):
   print(f"Refined Query: {q}")
   for i, m in enumerate(matches):
       ...  # print i+1, m.document.id, m.score, m.document.reranking_field
show_reranked(rerank_query, reranked.matches)


What to do: Complete the print logic to display each reranked note’s rank, ID, score, and the reranking_field.
Why: Allows you to compare how the reranker improves result ordering against the original search.


#Part 1: Load Documents & Execute Reranking Model

#1. Install Pinecone Libraries

Run this in your terminal or Colab/Jupyter notebook:

In [ ]:
!pip install pinecone==6.0.1 pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 6.5 MB/s eta 0:00:00


#2. Authenticate with Pinecone

In [ ]:
import os

if not os.environ.get("PINECONE_API_KEY"): from pinecone_notebooks.colab import Authenticate Authenticate() This prompts for your API key if it's not already set as an environment variable.

In [ ]:
from pinecone_notebooks.colab import Authenticate

In [ ]:
Authenticate()

#3. Instantiate the Pinecone Client

In [ ]:
from pinecone import Pinecone

In [ ]:
api_key = os.environ["PINECONE_API_KEY"]
environment = "Mystery" # e.g., "us-west1-gcp"
pc = Pinecone(api_key=api_key,
              environment=environment)

# What to do: Fill in your Pinecone project's environment string (found in your Pinecone dashboard) in place of .... Then create a Pinecone client instance. Why: The client (pc) is your entry point for all Pinecone operations—creating indexes, querying, and reranking.

#4. Define Your Query and Documents

In [ ]:
query = "Tell me about Apple's products"
documents = [ "Apple's latest iPhone features a new camera system.", "The apple tree in my backyard produces the sweetest fruit.", "Apple Inc. is known for its sleek hardware and intuitive software.", "Green apples are more sour than red ones.", "Apple recently announced new updates to the MacBook line." ]
# This mix helps the reranker distinguish between the company and the fruit.

#5. Call the Reranker

In [ ]:
from pinecone import RerankModel
reranked = pc.inference.rerank( model="bge-reranker-v2-m3", query=query,
                               documents=[{"id": str(i), "text": doc}
                                          for i, doc in enumerate(documents)], top_n=3 )
#This gets the top 3 most relevant documents.

#6. Inspect Reranked Results

In [ ]:
def show_reranked(query, matches):
  print(f"Query: {query}\n")
  for i, m in enumerate(matches):
    print(f"Rank {i+1}:")
    print(f" Score: {m.score:.4f}")
    print(f" Text: {m.document.text}\n")

In [ ]:
if reranked.matches:
  show_reranked(query, reranked.matches)
else:
  print("No reranked matches found.")

No reranked matches found.


This shows how relevant each document is according to the reranker.

#Part 2: Setup a Serverless Index for Medical Notes

Install data & model libraries

Install pandas for data manipulation, torch for model inference, and transformers for loading embedding models.

These libraries will make it possible to load, embed, and manipulate medical note data.

In [ ]:
!pip install pandas torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

Import modules & define environment settings

In [ ]:
import os

In [ ]:
import time

In [ ]:
import pandas as pd

In [ ]:
import torch

In [ ]:
from pinecone import Pinecone

In [ ]:
from pinecone import ServerlessSpec

In [ ]:
cloud = "AWS"        # e.g., "aws" - Replace with your cloud provider (e.g., "aws", "gcp", "azure")
region = "us-east-1"       # e.g., "us-east-1" - Replace with your Pinecone region
spec = ServerlessSpec(cloud=cloud, region=region) # Replace ... with appropriate CPU and memory_gb values
index_name = "pinecone-reranker"

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], environment=f"{cloud}-{region}")

What to do: Fill in cloud and region with your Pinecone project’s deployment environment. Choose CPU and memory values in ServerlessSpec. Why: You’re configuring a serverless index tailored to your resource requirements and connecting the client in the proper cloud region.

Create or recreate the index if pc.has_index(index_name): pc.delete_index(index_name) pc.create_index( name=index_name, dimension=..., # must match embedding vector size serverless_config=spec )

What to do: Set dimension equal to your embedding model’s output size (e.g., 384). Why: The index’s dimension must match the embedding vectors you’ll insert, otherwise upserts will fail.

In [ ]:
# No need to call pinecone.init() when using the Pinecone client object

index_name = "my-embeddings-index" # You can keep this index name or use "pinecone-reranker"

# Use the pc object to list, delete, and create indexes
if index_name in pc.list_indexes():
    pc.delete_index(index_name)

# For example, using MiniLM-L6-v2 which outputs 384-dim vectors
pc.create_index(
    name=index_name,
    dimension=384, # must match embedding vector size
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws", # Replace with your cloud provider
        region="us-east-1" # Replace with a region supported by your plan
    )
)

{
    "name": "my-embeddings-index",
    "metric": "cosine",
    "host": "my-embeddings-index-607ozbg.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

#Part 3: Load the Sample Data

Download & read JSONL import requests, tempfile

with tempfile.TemporaryDirectory() as tmpdir: file_path = os.path.join(tmpdir, "sample_notes_data.jsonl") url = "..." # raw GitHub URL to JSONL file resp = requests.get(url) resp.raise_for_status() open(file_path, "wb").write(resp.content) df = pd.read_json(file_path, orient='records', lines=True)

What to do: Insert the raw URL in place of ... to download the sample medical notes. Why: You need a DataFrame of medical notes (with embeddings already available) to index and test queries.



In [ ]:
import os
import requests
import tempfile
import pandas as pd

# Part 3: Load the Sample Data
with tempfile.TemporaryDirectory() as tmpdir:
    file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")

    # ✅ Replace this with a raw GitHub URL containing .jsonl data
    url = "https://raw.githubusercontent.com/pinecone-io/examples/master/docs/data/sample_notes_data.jsonl"

    # Download the file
    resp = requests.get(url)
    resp.raise_for_status()

    # Write content to a temp file
    with open(file_path, "wb") as f:
        f.write(resp.content)

    # Load into DataFrame
    df = pd.read_json(file_path, orient="records", lines=True)

Preview the DataFrame

In [ ]:
print(df.head())

     id                                             values  \
0  P011  [-0.2027486265, 0.2769146562, -0.1509393603, 0...   
1  P001  [0.1842793673, 0.4459365904, -0.0770567134, 0....   
2  P002  [-0.2040648609, -0.1739618927, -0.2897160649, ...   
3  P003  [0.1889383644, 0.2924542725, -0.2335938066, -0...   
4  P004  [-0.12171068040000001, 0.1674752235, -0.231888...   

                                            metadata  
0  {'advice': 'rest, hydrate', 'symptoms': 'heada...  
1  {'tests': 'EKG, stress test', 'symptoms': 'che...  
2  {'HbA1c': '7.2', 'condition': 'diabetes', 'med...  
3  {'symptoms': 'cough, wheezing', 'diagnosis': '...  
4  {'referral': 'dermatology', 'condition': 'susp...  


View the first few rows of the DataFrame. It ensures you have the right columns (e.g., id, embedding, metadata) before upserting.

In [ ]:
print(df.columns)

Index(['id', 'values', 'metadata'], dtype='object')


#Part 4: Upsert Data into the Index

Instantiate index client & upsert index

Instantiate index client & upsert
index = pc.Index(index_name) index.upsert_from_dataframe(df)

Create an Index object and call upsert_from_dataframe.

That process pushes all note embeddings and metadata into Pinecone for later queries.

In [ ]:
import time
from pinecone import Pinecone, ServerlessSpec

In [63]:
from pinecone import Pinecone
import pandas as pd
import time

# Initialize client
pc = Pinecone(api_key="pcsk_452ZGi_SxCLv9T4JtEHQVwg6PnYge5p8z9Yq6mzRqXwcryTvB3Rfrt5jbieQXcDQcesXN2")
index_name = "my-embeddings-index"

# Check if index exists
if index_name not in pc.list_indexes().names():
    raise ValueError(f"Index '{index_name}' does not exist. Please create it first.")

# Load the index
index = pc.Index(index_name)

# Load your DataFrame (assumed to be already loaded as `df`)
# df = pd.read_json(...) or however you loaded it

# Upsert from DataFrame
index.upsert_from_dataframe(df)

# Poll until vectors are indexed
def is_ready(idx):
    stats = idx.describe_index_stats()
    return stats.total_vector_count > 0

print("Waiting for upsert to complete...")
while not is_ready(index):
    time.sleep(5)
    print(index.describe_index_stats())

print("✅ Vectors upserted and ready for querying.")


sending upsert requests:   0%|          | 0/100 [00:00<?, ?it/s]

Waiting for upsert to complete...
{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100,
 'vector_type': 'dense'}
✅ Vectors upserted and ready for querying.



Part 4: Upsert Data into the Index

Instantiate index client & upsert
index = pc.Index(index_name) index.upsert_from_dataframe(df)

What to do: Create an Index object and call upsert_from_dataframe. Why: This pushes all your note embeddings and metadata into Pinecone for later queries.

Wait for availability
def is_ready(idx): stats = idx.describe_index_stats() return stats.total_vector_count > 0

while not is_ready(index): time.sleep(5) print(index.describe_index_stats())

What to do: Poll until total_vector_count is greater than zero. Why: Ensures that upserted vectors are fully indexed before you attempt to query.

#Part 5: Query & Embedding Function


Define an embedding function using sentence-transformers
Convert a user query into an embedding
Query the Pinecone index to get the most semantically similar results

In [64]:
!pip install sentence-transformers


Define your embedding function

In [65]:
from sentence_transformers import SentenceTransformer

# Load the model once (outside the function)
model = SentenceTransformer("all-MiniLM-L6-v2")  # Embedding dimension = 384

# Embedding function
def get_embedding(text):
    return model.encode(text)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Define a query and run semantic search

In [67]:
# Your clinical or semantic question
question = "what if my patient has leg pain?"

# Convert to embedding
emb = get_embedding(question)

# Run semantic search
results = index.query(
    vector=emb.tolist(),
    top_k=5,
    include_metadata=True
)

# Sort results by similarity score (descending)
matches = sorted(results.matches, key=lambda m: m.score, reverse=True)

# Display results
for match in matches:
    print(f"\nID: {match.id}")
    print(f"Score: {match.score:.4f}")
    print(f"Note: {match.metadata.get('note', '[no note]')}")



ID: P0100
Score: 0.5330
Note: [no note]

ID: P095
Score: 0.5082
Note: [no note]

ID: P047
Score: 0.5082
Note: [no note]

ID: P007
Score: 0.4544
Note: [no note]

ID: P092
Score: 0.4474
Note: [no note]


- "top_k", for number of results (e.g., 5), retrieves the most semantically similar notes from the index based on your clinical query.
- sentence_transformer "all-MiniLM-L6-v2" model is chosen; its role is  to convert incoming queries into the same vector space as your indexed notes.

#Part 6: Display & Rerank Clinical Notes


- Display initial semantic search results
- Prepare metadata for reranking
- Perform reranking using a refined query
- Display reranked results

1. Display initial search results

In [68]:
def show_results(q, matches):
    print(f"\n Initial Search Results for: {q}\n")
    for i, m in enumerate(matches):
        print(f"{i+1}. ID: {m.id}")
        print(f"   Score: {m.score:.4f}")
        print(f"   Metadata: {m.metadata}")


2. Prepare Documents for Reranking

In [69]:
# This summarizes all metadata fields into one string for reranking
rerank_docs = [
    {
        "id": m.id,
        "reranking_field": "; ".join([f"{k}: {v}" for k, v in m.metadata.items()])
    }
    for m in matches
]

# Refine the query to be more specific than the original one
rerank_query = "what symptoms are related to leg pain during walking?"


 3. Execute Reranking (Serverless)

 Choose top_n (e.g., 3 or 5) — the number of reranked items you want to see.

In [70]:
reranked = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=rerank_query,
    documents=rerank_docs,
    rank_fields=["reranking_field"],
    top_n=5  # or whatever number you want
)


 4. Display Reranked Results

In [71]:
def show_reranked(q, matches):
    print(f"\n🚀 Reranked Results for: {q}\n")
    for i, m in enumerate(matches):
        print(f"{i+1}. ID: {m.document.id}")
        print(f"   Score: {m.score:.4f}")
        print(f"   Content: {m.document.reranking_field}")


 Final Run

 Complete the print logic to display each reranked note’s rank, ID, score, and the reranking_field.
 It allows you to compare how the reranker improves result ordering against the original search.

In [76]:
show_results(question, matches)

if reranked.matches:
  show_reranked(rerank_query, reranked.matches)
else:
  print("No reranked matches found.")


 Initial Search Results for: what if my patient has leg pain?

1. ID: P0100
   Score: 0.5330
   Metadata: {'advice': 'over-the-counter pain relief, stretching', 'symptoms': 'muscle pain'}
2. ID: P095
   Score: 0.5082
   Metadata: {'symptoms': 'back pain', 'treatment': 'physical therapy'}
3. ID: P047
   Score: 0.5082
   Metadata: {'symptoms': 'back pain', 'treatment': 'physical therapy'}
4. ID: P007
   Score: 0.4544
   Metadata: {'surgery': 'knee arthroscopy', 'symptoms': 'pain, swelling', 'treatment': 'physical therapy'}
5. ID: P092
   Score: 0.4474
   Metadata: {'condition': 'dehydration', 'treatment': 'IV fluids'}
No reranked matches found.


# **As a conclusion:**

Reranking is used when higher precision is wanted, based on specific symptoms, treatments, or diagnoses